In [1]:
import cpp_soft_info

# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()

In [4]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibmq_mumbai'
DISTANCE = 11
ROUNDS = 11

md = metadata_loader(_extract=True, _drop_inutile=False)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["distance"] == DISTANCE]
md = md[md["backend_name"]==DEVICE]
md = md[md["rounds"]==ROUNDS]


md = md[md["meas_level"] == 1]
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
2576,2023-12-12 16:41:29.964000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7ya9gpnkg008kjdhg,NaN,{},[RepCode_Count_job],1.0,True,single,...,JobStatus.DONE,2023-12-12 16:47:11.376799+01:00,NaN,NaN,RepetitionCodeCircuit,3.0,11,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...
2575,2023-12-12 16:41:27.107000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7y9sgpnkg008kjdh0,NaN,{},[RepCode_Count_job],1.0,True,single,...,JobStatus.DONE,2023-12-12 16:46:53.325036+01:00,NaN,NaN,RepetitionCodeCircuit,3.0,11,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [5]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(30303, 25)


# Initialize the code

In [6]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid


layout = get_repcode_layout(distance=DISTANCE, backend=provider.get_backend(DEVICE), _is_hex=False)
qubit_mapping = get_repcode_IQ_map(layout, ROUNDS) #Hardcoded for repetition codes
print("layout:", layout)
print("len(layout):", len(layout), " = distance * code qubits + (distance-1) * link qubits")

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 34663.67it/s]

layout: [1, 3, 0, 2, 5]
len(layout): 5  = distance * code qubits + (distance-1) * link qubits


In [7]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-10T10:30:00" # "2023-11-22T" works to
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 40, 1.2, other_date=other_date)
# grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

Specified job execution date: 2023-12-12 16:46:53.325036+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-12-12 15:44:41.557972+00:00. Retrieving kde grid...
Searching for ibmq_mumbai and 23.12.12_15h41_40pts_1.2std


#### Noise model stim

In [8]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('cx', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [9]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {1: {'init': 0.011700000000000044, 'idle': 0.009264085822753843, 'RO': 0.011700000000000044, 'gate': 0.0001, '2-gate': {'default': 0.01, 0: 0.005124950217809426, 2: 0.009420430723780487}}, 3: {'init': 0.014100000000000001, 'idle': 0.012687261250991178, 'RO': 0.014100000000000001, 'gate': 0.0001, '2-gate': {'default': 0.01, 2: 0.004997507692207137, 5: 0.038329938870632824}}, 0: {'init': 0.021400000000000086, 'idle': 0.017300492940248713, 'RO': 0.021400000000000086, 'gate': 0.0001, '2-gate': {'default': 0.01, 1: 0.005124950217809426}}, 2: {'init': 0.018199999999999994, 'idle': 0.007659659995739809, 'RO': 0.018199999999999994, 'gate': 0.0001, '2-gate': {'default': 0.01, 1: 0.009420430723780487, 3: 0.004997507692207137}}, 5: {'init': 0.02499999999999991, 'idle': 0.009324692556279879, 'RO': 0.02499999999999991, 'gate': 0.0001, '2-gate': {'default': 0.01, 3: 0.038329938870632824}}}
avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]: [0.018080000000000006, 0.0112472

In [10]:
from soft_info import draw_matching_graph



circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0, #reset error,
                                before_measure_flip_probability= avgs_noise[2], #measurement error,
                                before_round_data_depolarization= 0)#avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)
# draw_matching_graph(matching, d, synd_rounds, figsize=(20, 20))   
# takes 10s

# Decode

In [11]:
from soft_info import get_counts, get_KDEs

counts = get_counts(memory)
print(counts)

18:28:13 Warning: Missing layout or synd_rounds, estimating outcomes without KDEs.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py, LINE: 67
18:28:13 Warning: Not enough kernels or no scaler provided. Using the magnitude of the real part for estimation... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py, LINE: 28
{'0000000000000000000000000': 16794, '0010000000000000000000000': 310, '1001000000000000000000000': 277, '1000000000000000000000000': 257, '1001000100000000000000000': 229, '1000010000000000000000000': 215, '1000010001000000000000000': 192, '1001000100010000000000000': 186, '1000010001000100000000000': 178, '0001010000000000000000000': 156, '0001010100000000000000000': 155, '0001010101000000000000000': 153, '0000101010101010101010101': 144, '0001000000000000000000000': 140, '1000010001000100010000000': 140, 

In [12]:
counts = cpp_soft_info.get_counts(memory[:], qubit_mapping, grid_dict, processed_scaler_dict, ROUNDS)
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
print(counts)

{'000 00 00 00 00 00 00 00 00 00 00 00': 16985, '100 00 00 00 00 00 00 00 00 00 00 00': 330, '100 10 00 00 00 00 00 00 00 00 00 00': 285, '100 10 00 10 00 00 00 00 00 00 00 00': 237, '100 00 10 00 00 00 00 00 00 00 00 00': 215, '100 00 10 00 10 00 00 00 00 00 00 00': 195, '100 10 00 10 00 10 00 00 00 00 00 00': 190, '100 00 10 00 10 00 10 00 00 00 00 00': 177, '000 10 10 00 00 00 00 00 00 00 00 00': 160, '000 10 10 10 10 00 00 00 00 00 00 00': 160, '000 10 10 10 00 00 00 00 00 00 00 00': 150, '000 01 01 01 01 01 01 01 01 01 01 01': 147, '000 10 00 00 00 00 00 00 00 00 00 00': 141, '100 00 10 00 10 00 10 00 10 00 00 00': 141, '000 10 10 10 10 10 10 00 00 00 00 00': 139, '100 10 00 10 00 10 00 10 00 10 00 00': 133, '000 10 10 10 10 10 00 00 00 00 00 00': 132, '100 10 00 10 00 10 00 10 00 00 00 00': 131, '000 10 10 10 10 10 10 10 10 10 10 10': 114, '000 10 10 10 10 10 10 10 00 00 00 00': 110, '000 10 10 10 10 10 10 10 10 00 00 00': 106, '000 01 00 00 00 00 00 00 00 00 00 00': 105, '000 01

In [13]:
matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-3 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory,
                                           ROUNDS, qubit_mapping, grid_dict,
                                           processed_scaler_dict, p_data=-1, p_mixed=-1, #p_mixed=1e-80, for d=30 
                                           common_measure=-1, _bimodal=False, merge_strategy = "replace")
print("num_errors:", num_errors, "out of", len(memory), "shots")
 
# takes 5s

num_errors: 644 out of 30303 shots


In [14]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory,
                                           ROUNDS, qubit_mapping, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

num_errors: 631 out of 30303 shots


In [15]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


p_data = -1
p_mixed = -1
p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           ROUNDS, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(memory), "shots")
         
# takes 1s

num_errors: 574 out of 30303 shots


# Decode mumbai code job

In [16]:
md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["distance"] == DISTANCE]
md = md[md["backend_name"]==DEVICE]
md = md[md["rounds"]==ROUNDS]


md = md[md["meas_level"] == 0] # for count job
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
2584,2023-12-12 16:42:41.296000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7ywbwsx00008tfm50,[RepCode_Count_job_count],0.0,30303.0,NaN,NaN,NaN,...,JobStatus.DONE,2023-12-12 16:50:02.744185+01:00,NaN,NaN,RepetitionCodeCircuit,3.0,11,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...
2583,2023-12-12 16:42:38.357000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7yvkvn4c0008a7rfg,[RepCode_Count_job_count],0.0,30303.0,NaN,NaN,NaN,...,JobStatus.DONE,2023-12-12 16:49:52.045187+01:00,NaN,NaN,RepetitionCodeCircuit,3.0,11,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [17]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_counts()
    jobs[mmr_name] = job_id

counts = memories[memory_str]
job_id = jobs[memory_str]

#sort the counts dict
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
print(counts)

{'000 00 00 00 00 00 00 00 00 00 00 00': 14434, '100 10 00 00 00 00 00 00 00 00 00 00': 379, '100 00 10 00 00 00 00 00 00 00 00 00': 339, '100 00 10 00 10 00 00 00 00 00 00 00': 314, '100 00 00 00 00 00 00 00 00 00 00 00': 312, '001 00 00 00 00 00 00 00 00 00 00 00': 301, '100 10 00 10 00 00 00 00 00 00 00 00': 292, '100 10 00 10 00 10 00 00 00 00 00 00': 265, '100 00 10 00 10 00 10 00 00 00 00 00': 247, '100 10 00 10 00 10 00 10 00 00 00 00': 224, '100 00 10 00 10 00 10 00 10 00 00 00': 209, '100 00 10 00 10 00 10 00 10 00 10 00': 193, '100 10 00 10 00 10 00 10 00 10 00 00': 185, '000 10 10 10 00 00 00 00 00 00 00 00': 166, '000 10 10 00 00 00 00 00 00 00 00 00': 157, '000 10 00 00 00 00 00 00 00 00 00 00': 155, '000 10 10 10 10 00 00 00 00 00 00 00': 137, '000 10 10 10 10 10 00 00 00 00 00 00': 114, '000 10 10 10 10 10 10 00 00 00 00 00': 114, '000 01 01 01 01 01 01 01 01 01 01 01': 110, '000 01 00 00 00 00 00 00 00 00 00 00': 110, '000 10 10 10 10 10 10 10 00 00 00 00': 104, '000 10

In [18]:
from soft_info import counts_to_det_syndr
from tqdm import tqdm

matching = pymatching.Matching.from_detector_error_model(model)
cpp_soft_info.reweight_edges_to_one(matching._matching_graph)

num_errors = 0
for count_key, shots in tqdm(zip(counts.keys(), counts.values())):
    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    actual_observables = [(int(count_key[0])+0)%2]

    if actual_observables != predicted_observables:
        num_errors += shots

print(f"num_errors: {num_errors} out of {sum(counts.values())} shots")

0it [00:00, ?it/s]

4092it [00:00, 30795.59it/s]

num_errors: 840 out of 30303 shots
